# Hybrid Quantum-Classical Medical Imaging on Colab

This notebook sets up the environment to run the QML medical imaging project directly from GitHub using Google Colab.


In [ ]:
# 1. Setup Environment & Mount Drive
import os
import shutil
from google.colab import drive
from getpass import getpass

# Mount Drive
drive.mount('/content/drive')

# Configuration
REPO_URL = "https://github.com/DreamX55/QML-for-Medical-Imaging.git"
PROJECT_NAME = "qml_medical_imaging"  # Local folder name
PROJECT_ROOT = f"/content/{PROJECT_NAME}"

# --- CUSTOMIZE THESE PATHS IF NEEDED ---
DRIVE_DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset'
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/qml_medical_imaging_outputs'
# ---------------------------------------


In [ ]:
# 2. Check Dataset Existence
if not os.path.exists(DRIVE_DATASET_PATH):
    print(f"❌ Error: Dataset not found at {DRIVE_DATASET_PATH}")
    print("Please make sure you uploaded the 'brain_tumor_dataset' folder to 'medical_imaging_data' in your Google Drive.")
    # List parent dir to help debug
    parent = os.path.dirname(DRIVE_DATASET_PATH)
    if os.path.exists(parent):
        print(f"Contents of {parent}:")
        print(os.listdir(parent))
    else:
        print(f"Parent directory {parent} does not exist either.")
    raise FileNotFoundError("Dataset not found in Google Drive")
else:
    print(f"✅ Dataset found at {DRIVE_DATASET_PATH}")

In [ ]:
# 3. Clone/Update Repository
if os.path.exists(PROJECT_ROOT):
    print(f"Repo exists at {PROJECT_ROOT}. Pulling latest changes...")
    %cd {PROJECT_ROOT}
    !git pull
else:
    print(f"Cloning repo to {PROJECT_ROOT}...")
    %cd /content
    # Handle Private Repo
    try:
        !git clone {REPO_URL} {PROJECT_NAME}
    except:
        print("Public clone failed. Assuming private repo...")
        token = getpass('Enter GitHub PAT: ')
        user = REPO_URL.split('/')[-2]
        repo_name = REPO_URL.split('/')[-1]
        !git clone https://{token}@github.com/{user}/{repo_name} {PROJECT_NAME}

%cd {PROJECT_ROOT}
!pip install -r requirements.txt

In [ ]:
# 4. Link Data & Outputs
data_link = os.path.join(PROJECT_ROOT, 'data/brain_mri')
output_link = os.path.join(PROJECT_ROOT, 'outputs')

def force_symlink(src, dst):
    if os.path.exists(dst) or os.path.islink(dst):
        if os.path.isdir(dst) and not os.path.islink(dst):
            shutil.rmtree(dst)
        else:
            os.remove(dst)
    os.symlink(src, dst)
    print(f"Linked {src} -> {dst}")

os.makedirs(os.path.join(PROJECT_ROOT, 'data'), exist_ok=True)

# Link Data
force_symlink(DRIVE_DATASET_PATH, data_link)

# Link Outputs
if not os.path.exists(DRIVE_OUTPUT_PATH):
    os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)
force_symlink(DRIVE_OUTPUT_PATH, output_link)

In [ ]:
# 5. Run Training
%cd {PROJECT_ROOT}

!python main.py --mode train \
    --data_dir data/brain_mri \
    --model_type hybrid \
    --n_qubits 10 \
    --n_layers 2 \
    --entanglement ring \
    --epochs 30 \
    --batch_size 16 \
    --output_dir outputs